In [1]:
import sys
sys.path.append('../RecSysRep/')

In [2]:
import Basics.Load as ld
import scipy.sparse as sps

URM_all, ICM1, ICM2, ICM3, ICM4 = ld.getCOOs()
ICML = ld.getICMlength('5km')
ICM_weighted = ld.getICMselected('7')
ICM_gen = sps.hstack([ICM1, ICM2])
ICM_all = sps.hstack([ICM1, ICM2, ICM3, ICML])
# URM_train, URM_val = ld.getSplit(URM_train_val, 5678, 0.8)

In [3]:
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8, seed=1234)
# URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.8)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
# evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 13643 ( 0.1%) Users that have less than 1 test interactions


In [13]:
import os

ofp = "../models_temp/Similarity_Hybrid/New/"

models_to_combine_best = {
                            'EASE': {"topK": None, "normalize_matrix": False, "l2_norm": 2712.02456},
                            'IALS_Hyb' : {"num_factors": 28, "epochs": 10, "confidence_scaling": "linear", "alpha": 0.43657990940994623, "epsilon": 0.35472063248578317, "reg": 0.0001698292271931609, "mw": 0.06122362507952762},
                            'RP3ICMnew': {'alpha': 1.029719677583138, 'beta': 1.0630164752134375, 'topK': 6964, 'normalize_similarity': True},
                            'RP3ICM' : {"topK": 2550, "alpha": 1.3058102610510849, "beta": 0.5150718337969987, "normalize_similarity": True, "implicit": True},
                            'IALS' :  {'num_factors': 34, 'epochs': 599, 'confidence_scaling': 'linear', 'alpha': 0.003519435539271083, 'epsilon': 0.09222402080721787, 'reg': 2.4127708108457617e-05},
                            'SLIMgensub': {"l1_ratio" : 0.025887359156206147, "topK": 2140, "alpha": 0.009567288586539689, "workers": 8, "mw": 1},
                            'SLIMBPR' : {"epochs": 440, "lambda_i": 0.007773815998802306, "lambda_j": 0.003342522366982381, "learning_rate": 0.010055161410725193, "topK": 4289, "random_seed": 1234, "sgd_mode": "sgd"},
                            'SLIMweig': {'l1_ratio': 0.0005247075138160404, 'topK': 4983, 'alpha': 0.06067400905430761, 'workers': 8, 'mw': 2.308619939318322},
                            'SLIMER': {'topK': 6000, 'l1_ratio': 0.0005495104968035837, 'alpha': 0.08007142704041009, 'workers': 8},
                            'P3alpha': {'topK': 4834, 'alpha': 1.764994849187595, 'normalize_similarity': True, 'implicit': True},
                            'RP3beta': {"topK": 1049, "alpha": 1.1626473723475605, "beta": 0.6765017195261293, "normalize_similarity": True, "implicit": True},
                            'icm_all' : {"shrink": 5675, "topK": 2310, "feature_weighting": "BM25", "normalize": False},
                            'icm_weighted': {"shrink": 4000, "topK": 985, "feature_weighting": "TF-IDF", "normalize": True},
                            'SLIM_all' : {'l1_ratio': 0.0001043005302985496, 'topK': 1135, 'alpha': 0.06322495726710943, 'workers': 8, 'mw': 1.2937257828415842},
                            'UserKNN' : {"topK": 469, "similarity": "cosine", "shrink": 588, "normalize": True, "feature_weighting": "TF-IDF", "URM_bias": False},
                            'SLIM_BPR_Hyb' : {"epochs": 1443, "lambda_i": 8.900837513818856e-05, "lambda_j": 1.2615223007492727e-05, "learning_rate": 0.0037706733838839264, "topK": 6181, "random_seed": 1234, "sgd_mode": "sgd"},
                            'Neural': {"epochs": 295, "learning_rate": 9.598708662230564e-05, "l2_reg": 1.4122381520119457e-05, "dropout": 0.5759445339123964, "total_anneal_steps": 147387, "anneal_cap": 0.4376000433422989, "batch_size": 256, "encoding_size": 185, "next_layer_size_multiplier": 6, "max_n_hidden_layers": 1, "max_layer_size": 5000.0}
                         }


In [5]:
def model_init(recommender, name, args):
    path_name = ofp + name + ".zip"
    print(path_name)
    if os.path.exists(path_name):
        print("Model found!")
        recommender.load_model(ofp, name)
    else:
        print("Model does not exists, creating...")
        if not os.path.exists(ofp):
            print("Main folder does not exist, creating...")
            os.makedirs(ofp)
        recommender.fit(**args)
        recommender.save_model(ofp, name)

In [6]:
from Recommenders.NonPersonalizedRecommender import TopPop
from Recommenders.KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython_Hybrid
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIM_S_ElasticNetRecommender
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommenderICM
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender_Hybrid
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Recommenders.Neural.MultVAERecommender import MultVAERecommender

In [7]:
from Recommenders.KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender
from Recommenders.KNN.ItemKNNSimilarityHybridRecommenderNormal import ItemKNNSimilarityHybridRecommenderNormal
from Recommenders.KNN.ItemKNNScoresHybridMultipleRecommender import ItemKNNScoresHybridMultipleRecommender
from Recommenders.KNN.ItemKNNScoresHybridMultipleRecommender import ItemKNNScoresHybridTwoRecommender
from Recommenders.HybridScores.DifferentStructure import TwoDifferentModelRecommender

In [14]:
recommender1 = MultiThreadSLIM_SLIM_S_ElasticNetRecommender(URM_train, ICM_all)
recommender2 = EASE_R_Recommender(URM_train)
recommender3 = ItemKNNCBFRecommender(URM_train, ICM_all)

model_init(recommender1, 'SLIMall', models_to_combine_best['SLIM_all'])
model_init(recommender2, 'EASE', models_to_combine_best['EASE'])
model_init(recommender3, 'ItemKNNall', models_to_combine_best['icm_all'])

../models_temp/Similarity_Hybrid/New/SLIMall.zip
Model found!
SLIMElasticNetRecommender: Loading model from file '../models_temp/Similarity_Hybrid/New/SLIMall'
SLIMElasticNetRecommender: Loading complete
../models_temp/Similarity_Hybrid/New/EASE.zip
Model found!
EASE_R_Recommender: Loading model from file '../models_temp/Similarity_Hybrid/New/EASE'
EASE_R_Recommender: Loading complete
../models_temp/Similarity_Hybrid/New/ItemKNNall.zip
Model does not exists, creating...
Similarity column 18059 (100.0%), 1529.42 column/sec. Elapsed time 11.81 sec
ItemKNNCBFRecommender: Saving model in file '../models_temp/Similarity_Hybrid/New/ItemKNNall'
ItemKNNCBFRecommender: Saving complete


In [15]:
def rank_models(evaluator, recommenders):
    for r in recommenders:
        r_d, _ = evaluator.evaluateRecommender(r)
        print(r.RECOMMENDER_NAME, r_d.loc[10]['MAP'])

In [23]:
rank_models(evaluator_validation, [recommender_knn_slim, recommender2, recommender3])

EvaluatorHoldout: Processed 13643 (100.0%) in 27.07 sec. Users per second: 504
             MAP
cutoff          
10      0.249891
ItemKNNSimilarityHybridRecommender 0.24989120532769263


KeyboardInterrupt: 

In [17]:
import numpy as np

def test_percentage_sim(recommender_a, recommender_b, evaluator, high1, low1, step):
    recommender = ItemKNNSimilarityHybridRecommenderNormal(URM_train, recommender_a.W_sparse, recommender_b.W_sparse)
    results = []
    alp_space = np.linspace(high1, low1, step, True)
    for alp in alp_space:
        recommender.fit(alp)
        r_d, _ = evaluator.evaluateRecommender(recommender)
        print(alp, ":", r_d.loc[10]['MAP'])
        results.append(r_d.loc[10]['MAP'])
    
    return alp_space, results

def test_percentage_sco(recommender_a, recommender_b, evaluator, high1, low1, step, norm=None):
    recommender = TwoDifferentModelRecommender(URM_train, recommender_a, recommender_b)
    results = []
    alp_space = np.linspace(high1, low1, step, True)
    for alp in alp_space:
        recommender.fit(norm, alp)
        r_d, _ = evaluator.evaluateRecommender(recommender)
        print(alp, ":", r_d.loc[10]['MAP'])
        results.append(r_d.loc[10]['MAP'])
    
    return alp_space, results

In [21]:
alp_space, results = test_percentage_sim(recommender1, recommender3, evaluator_validation, 0.895, 0.906, 8)

EvaluatorHoldout: Processed 13643 (100.0%) in 49.02 sec. Users per second: 278
             MAP
cutoff          
10      0.249781
0.895 : 0.24978093306760019
EvaluatorHoldout: Processed 13643 (100.0%) in 27.38 sec. Users per second: 498
             MAP
cutoff          
10      0.249836
0.8965714285714286 : 0.24983634260955961
EvaluatorHoldout: Processed 13643 (100.0%) in 27.79 sec. Users per second: 491
             MAP
cutoff          
10      0.249844
0.8981428571428571 : 0.24984446061646806
EvaluatorHoldout: Processed 13643 (100.0%) in 28.03 sec. Users per second: 487
             MAP
cutoff          
10      0.249891
0.8997142857142857 : 0.24989120532769263
EvaluatorHoldout: Processed 13643 (100.0%) in 28.82 sec. Users per second: 473
             MAP
cutoff          
10      0.249881
0.9012857142857144 : 0.24988131596062388
EvaluatorHoldout: Processed 13643 (100.0%) in 27.79 sec. Users per second: 491
             MAP
cutoff          
10      0.249812
0.9028571428571429 : 0.24981

In [22]:
best_alp = 0.8997142857142857
recommender_knn_slim = ItemKNNSimilarityHybridRecommenderNormal(URM_train, recommender1.W_sparse, recommender3.W_sparse)
recommender_knn_slim.fit(best_alp)

In [29]:
alp_space, results = test_percentage_sco(recommender_knn_slim, recommender2, evaluator_validation, 0.4, 0.6, 20) # 0.6305084745762711 : 0.250522906101042

EvaluatorHoldout: Processed 13643 (100.0%) in 1.52 min. Users per second: 150
             MAP
cutoff          
10      0.250344
0.4 : 0.25034446410683264
EvaluatorHoldout: Processed 13643 (100.0%) in 1.49 min. Users per second: 153
             MAP
cutoff          
10      0.250347
0.4105263157894737 : 0.2503470149818089
EvaluatorHoldout: Processed 13643 (100.0%) in 1.55 min. Users per second: 147
             MAP
cutoff          
10      0.250303
0.4210526315789474 : 0.25030319054250705
EvaluatorHoldout: Processed 13643 (100.0%) in 1.48 min. Users per second: 154
            MAP
cutoff         
10      0.25028
0.43157894736842106 : 0.25027964803160874
EvaluatorHoldout: Processed 13643 (100.0%) in 1.49 min. Users per second: 153
             MAP
cutoff          
10      0.250444
0.4421052631578948 : 0.25044414601824994
EvaluatorHoldout: Processed 13643 (100.0%) in 1.49 min. Users per second: 152
             MAP
cutoff          
10      0.250472
0.45263157894736844 : 0.250471911870614

KeyboardInterrupt: 

In [30]:
ofp = "../all_models/URM_all/"

recommender1a = MultiThreadSLIM_SLIM_S_ElasticNetRecommender(URM_all, ICM_all)
recommender2a = EASE_R_Recommender(URM_all)
recommender3a = ItemKNNCBFRecommender(URM_all, ICM_all)

model_init(recommender1a, 'SLIMall', models_to_combine_best['SLIM_all'])
model_init(recommender2a, 'EASE', models_to_combine_best['EASE'])
model_init(recommender3a, 'ItemKNNall', models_to_combine_best['icm_all'])

../all_models/URM_all/SLIMall.zip
Model found!
SLIMElasticNetRecommender: Loading model from file '../all_models/URM_all/SLIMall'
SLIMElasticNetRecommender: Loading complete
../all_models/URM_all/EASE.zip
Model found!
EASE_R_Recommender: Loading model from file '../all_models/URM_all/EASE'
EASE_R_Recommender: Loading complete
../all_models/URM_all/ItemKNNall.zip
Model does not exists, creating...
Similarity column 18059 (100.0%), 1381.26 column/sec. Elapsed time 13.07 sec
ItemKNNCBFRecommender: Saving model in file '../all_models/URM_all/ItemKNNall'
ItemKNNCBFRecommender: Saving complete


In [35]:
best_alp = 0.8997142857142857
recommender_knn_slima = ItemKNNSimilarityHybridRecommenderNormal(URM_all, recommender1a.W_sparse, recommender3a.W_sparse)
recommender_knn_slima.fit(best_alp)

In [36]:
recommender_final = TwoDifferentModelRecommender(URM_all, recommender_knn_slima, recommender2a)
recommender_final.fit(None, 0.49473684210526314)

In [37]:
import pandas as pd
K=10

user_test_path = '../data/data_target_users_test.csv'
user_test_dataframe = pd.read_csv(filepath_or_buffer=user_test_path,
                                sep=",",
                                dtype={0:int})

subm_set = user_test_dataframe.to_numpy().T[0]


subm_res = {"user_id":[], "item_list":[]}

for user_id in subm_set:
    subm_res["user_id"].append(user_id)
    res = recommender_final.recommend(user_id, K)
    print("USER", user_id, "RECOMMENDED!")
    res = ' '.join(map(str, res))
    print(res)
    subm_res["item_list"].append(res)


# print(subm_res)

submission = pd.DataFrame.from_dict(subm_res)
# submission

from datetime import datetime
now = datetime.now() # current date and time


submission.to_csv('../subs/submission {:%Y_%m_%d %H_%M_%S}.csv'.format(now), index=False)

USER 0 RECOMMENDED!
3899 5433 16365 6061 16359 8465 640 2243 9130 10586
USER 1 RECOMMENDED!
3473 14400 940 16012 9275 16365 2118 640 10118 11662
USER 2 RECOMMENDED!
6177 1551 17022 17330 16365 14229 15119 2748 5925 4607
USER 3 RECOMMENDED!
8194 15551 16359 15082 960 8371 5885 6118 7143 16444
USER 4 RECOMMENDED!
8194 10586 10954 2152 9871 16494 5299 13691 241 6177
USER 5 RECOMMENDED!
1846 14396 16640 9621 7867 11227 13098 13585 99 8852
USER 6 RECOMMENDED!
6214 985 16846 14358 14181 8207 7606 9621 14868 13443
USER 7 RECOMMENDED!
13963 15119 1885 3433 4292 14031 7751 17364 640 3768
USER 8 RECOMMENDED!
8635 3619 5605 1259 2665 10865 5299 3768 8502 6011
USER 9 RECOMMENDED!
5605 8207 1614 13318 14181 14078 1551 16755 15117 3473
USER 10 RECOMMENDED!
14358 3899 6214 4607 14400 13229 6975 16365 15181 2243
USER 11 RECOMMENDED!
14181 10865 3473 11359 6431 12390 249 5577 1836 5299
USER 12 RECOMMENDED!
3899 6724 3200 1137 1792 13963 15778 5925 13134 1619
USER 13 RECOMMENDED!
8635 6214 15119 6975 14

USER 124 RECOMMENDED!
3899 16946 7744 14181 1259 1551 2665 12186 1137 17135
USER 125 RECOMMENDED!
15750 13134 6905 3899 17936 14181 5273 1836 10295 3889
USER 126 RECOMMENDED!
3473 11339 14181 17748 5433 3829 6177 7295 17961 960
USER 127 RECOMMENDED!
5141 15922 2687 6676 16759 7003 10776 17944 13235 873
USER 128 RECOMMENDED!
6975 17364 9131 14181 249 16946 10295 1614 7975 6589
USER 129 RECOMMENDED!
6214 10865 16365 12707 640 5273 8011 15119 14181 8194
USER 130 RECOMMENDED!
17330 14181 3829 1551 11339 4204 13032 14213 16598 17388
USER 131 RECOMMENDED!
2772 16946 7744 10295 16808 3768 9131 17788 3473 2665
USER 132 RECOMMENDED!
3899 13134 12707 1259 1137 16365 1551 8651 985 15119
USER 133 RECOMMENDED!
6157 8011 5433 8473 13963 6177 3768 985 7975 7374
USER 134 RECOMMENDED!
2243 15583 16846 11844 12943 2665 17330 12708 13032 6011
USER 135 RECOMMENDED!
7264 3910 3200 1259 17135 9871 11854 8194 15750 5273
USER 136 RECOMMENDED!
4088 11702 3200 4093 2680 840 7407 11776 10295 12028
USER 137 RECOM

17356 14709 3473 3910 6177 8207 3768 11391 5925 12752
USER 238 RECOMMENDED!
8635 10865 512 3768 1614 7295 640 4582 9871 16846
USER 239 RECOMMENDED!
15922 7744 14358 12901 17294 8096 14037 16143 5640 14798
USER 240 RECOMMENDED!
3899 3619 6214 6652 16365 5273 1551 2665 3942 14078
USER 241 RECOMMENDED!
6177 8194 15119 11430 16962 8635 7606 4582 13229 14781
USER 242 RECOMMENDED!
17135 3899 2756 3942 14627 14400 13638 3200 13691 1259
USER 243 RECOMMENDED!
1619 3899 15181 4582 7295 2886 12901 8455 17936 6961
USER 244 RECOMMENDED!
13654 3364 5403 1208 9776 13318 9441 4883 11339 16038
USER 245 RECOMMENDED!
6214 16365 14181 1614 14358 249 3942 1137 17416 4173
USER 246 RECOMMENDED!
2376 13654 7716 8635 11844 14266 5186 14868 14181 9322
USER 247 RECOMMENDED!
9871 13638 5403 5424 5974 12663 1386 5605 17526 9776
USER 248 RECOMMENDED!
8502 10865 13725 12186 17294 6011 640 552 17788 6589
USER 249 RECOMMENDED!
1232 11430 5675 6214 16365 2301 3473 6016 3080 552
USER 250 RECOMMENDED!
6214 6975 14266 108

USER 358 RECOMMENDED!
6214 7645 6652 5300 3768 8194 3899 15181 9308 2665
USER 359 RECOMMENDED!
3899 16365 16791 1137 14509 4853 16486 17416 16946 4204
USER 360 RECOMMENDED!
5273 1551 16486 10370 2126 3942 7309 10562 15469 17330
USER 361 RECOMMENDED!
14358 3473 640 6652 249 12901 17330 17364 14181 16624
USER 362 RECOMMENDED!
6177 3899 13229 16365 249 8635 12943 640 14358 14400
USER 363 RECOMMENDED!
2665 5324 15181 8635 12901 1846 1619 13032 13229 1941
USER 364 RECOMMENDED!
13410 12901 17788 17356 16946 16791 5640 17135 13134 6975
USER 365 RECOMMENDED!
1551 15778 15551 17135 15683 16486 15844 16070 1208 5981
USER 366 RECOMMENDED!
13410 5273 6214 8635 5925 3473 10370 1259 17330 16365
USER 367 RECOMMENDED!
12752 9131 3200 6177 2182 3829 7870 13410 6118 12492
USER 368 RECOMMENDED!
6177 9621 552 12764 8473 99 1259 2796 53 5299
USER 369 RECOMMENDED!
3899 5273 5742 2756 12390 7407 14400 5424 16278 14358
USER 370 RECOMMENDED!
17022 16598 11339 3899 17584 6306 6177 7407 1515 3200
USER 371 RECOMM

USER 483 RECOMMENDED!
8207 9131 5974 13963 17584 1137 4204 16365 16846 8011
USER 484 RECOMMENDED!
8635 4204 2756 5974 17330 7099 287 13914 15119 15996
USER 485 RECOMMENDED!
12752 6214 10865 16640 14868 15868 7606 11504 15289 4582
USER 486 RECOMMENDED!
3768 5324 14358 13410 2665 6011 14181 13725 5273 16364
USER 487 RECOMMENDED!
3889 5141 4078 6214 17670 17330 1885 1725 13914 640
USER 488 RECOMMENDED!
12457 17330 6214 13134 8326 1208 7716 14868 13178 16486
USER 489 RECOMMENDED!
6540 11303 202 17159 8316 14813 11576 1838 7089 6214
USER 490 RECOMMENDED!
3899 14400 6214 15750 241 1551 16791 17022 1259 17364
USER 491 RECOMMENDED!
12492 7309 13318 3899 1259 12311 3473 17943 3942 7429
USER 492 RECOMMENDED!
8898 1838 3204 2665 840 13950 13134 5675 7983 2925
USER 493 RECOMMENDED!
7716 3473 13654 14868 9776 1551 8651 103 4088 12943
USER 494 RECOMMENDED!
7224 17330 6782 249 13134 17135 13638 13914 17526 8747
USER 495 RECOMMENDED!
15119 11339 17584 14358 6905 1941 640 8326 1614 8247
USER 496 RECOMM

...

USER 13072 RECOMMENDED!
3899 2152 16791 5273 12707 12943 5324 17462 4853 13178
USER 13073 RECOMMENDED!
11240 2772 7141 15750 15558 2456 11491 2748 10865 4088
USER 13074 RECOMMENDED!
17330 6975 2665 4582 419 10954 6589 16549 16791 5433
USER 13075 RECOMMENDED!
8635 7295 1386 249 1551 15969 129 2851 13178 13638
USER 13076 RECOMMENDED!
6177 16946 8207 3899 17135 14358 3889 6975 17330 7867
USER 13077 RECOMMENDED!
8567 1137 11359 11342 5751 16365 7412 16070 1551 4721
USER 13078 RECOMMENDED!
6214 6177 9621 5299 6975 6652 13410 11227 8207 7606
USER 13079 RECOMMENDED!
1519 15596 7744 99 9274 1246 8635 10430 6503 11349
USER 13080 RECOMMENDED!
5925 13134 13229 7983 15778 13410 1259 10316 1941 8936
USER 13081 RECOMMENDED!
12707 17788 3768 5925 5299 17330 3200 17294 5273 1551
USER 13082 RECOMMENDED!
5403 5925 14400 17135 9441 17936 512 13134 15969 17961
USER 13083 RECOMMENDED!
1551 14396 3080 10370 5273 3829 3198 17578 2301 16486
USER 13084 RECOMMENDED!
7744 6177 14010 13296 17788 3200 2076 5974 11

USER 13207 RECOMMENDED!
16070 1941 16364 8747 14266 6214 2772 7295 6011 13032
USER 13208 RECOMMENDED!
16846 7374 17294 4078 6177 1044 13229 2886 7983 1137
USER 13209 RECOMMENDED!
16365 3473 2665 17830 14358 8635 1619 4582 5324 10865
USER 13210 RECOMMENDED!
16549 15082 16365 10010 14396 6214 15154 3473 6975 17330
USER 13211 RECOMMENDED!
12975 15119 4559 5273 8207 11491 12186 17670 13131 8635
USER 13212 RECOMMENDED!
6214 17788 5885 5433 1614 1137 17294 13229 3955 4292
USER 13213 RECOMMENDED!
10243 12752 16365 5951 5273 11430 14181 1551 16846 14358
USER 13214 RECOMMENDED!
9441 1653 5925 6676 15969 241 17022 1259 12390 5742
USER 13215 RECOMMENDED!
6177 14358 15119 4582 16365 5403 8635 17330 6214 14868
USER 13216 RECOMMENDED!
5273 1551 2665 1137 13914 2772 5925 7099 5424 3889
USER 13217 RECOMMENDED!
12901 6011 1792 12186 13410 8635 3768 12752 17140 17330
USER 13218 RECOMMENDED!
6157 13295 5951 5640 12832 3473 7374 6214 16365 7606
USER 13219 RECOMMENDED!
3869 3200 14078 7973 11776 6214 9398 

USER 13331 RECOMMENDED!
17294 13549 6112 15119 507 1267 16365 5925 12708 10954
USER 13332 RECOMMENDED!
11303 3473 13950 7264 15383 9527 13914 11873 14358 14813
USER 13333 RECOMMENDED!
8194 13229 16359 15551 6177 12901 2243 15583 6118 2182
USER 13334 RECOMMENDED!
5925 2665 1259 15119 2820 15750 12663 241 8207 5424
USER 13335 RECOMMENDED!
17416 2886 1792 13410 6975 8635 17788 16549 14781 12703
USER 13336 RECOMMENDED!
13134 17330 15750 6177 2387 13981 14400 15969 6214 17388
USER 13337 RECOMMENDED!
6214 15181 3473 5299 15119 17294 5273 15969 13914 8455
USER 13338 RECOMMENDED!
640 16791 16365 8651 5324 3200 16364 17462 15683 7309
USER 13339 RECOMMENDED!
5324 5885 17364 7143 985 2243 4582 15868 16444 15583
USER 13340 RECOMMENDED!
13638 640 7606 3473 14868 11359 6177 11836 17022 13134
USER 13341 RECOMMENDED!
3899 17416 8726 4961 16791 4619 1614 1725 17961 2846
USER 13342 RECOMMENDED!
8326 6670 7412 3899 2680 17411 10370 14627 15750 13134
USER 13343 RECOMMENDED!
13914 17330 16364 3473 1137 407

USER 13453 RECOMMENDED!
8207 13410 3899 7716 12943 17364 15181 13032 640 8502
USER 13454 RECOMMENDED!
6177 8635 6214 14358 4582 5925 15119 1619 17788 1941
USER 13455 RECOMMENDED!
13229 1941 14400 5605 14181 4853 4582 1614 13178 11481
USER 13456 RECOMMENDED!
8635 5925 14396 15778 17584 15969 241 12663 7309 11339
USER 13457 RECOMMENDED!
16365 2665 16846 8207 5273 6011 5605 17330 5925 1551
USER 13458 RECOMMENDED!
14181 5925 8635 5273 16365 2665 6905 1259 14358 17748
USER 13459 RECOMMENDED!
3473 6975 3768 1259 14358 6011 12186 7716 17294 5299
USER 13460 RECOMMENDED!
15750 16364 9871 3473 2851 3569 1653 13725 3899 5424
USER 13461 RECOMMENDED!
15119 5273 13387 6214 13914 8326 10865 7486 12457 14781
USER 13462 RECOMMENDED!
8207 15119 3619 3889 13914 10316 8573 13134 1515 17330
USER 13463 RECOMMENDED!
3473 3899 16365 1792 14181 17330 15119 6011 1137 5925
USER 13464 RECOMMENDED!
9871 15969 8037 12663 16364 15778 7295 12378 3899 11553
USER 13465 RECOMMENDED!
5273 6214 7870 3619 13410 12492 17584

USER 13559 RECOMMENDED!
16846 12901 3473 3899 10010 6011 13410 4582 17623 15922
USER 13560 RECOMMENDED!
6652 6157 15119 11844 16365 16846 10316 9898 2482 2796
USER 13561 RECOMMENDED!
6214 13387 17356 6011 2886 9871 4865 4721 1619 14781
USER 13562 RECOMMENDED!
5403 17330 14181 8635 7099 4088 17961 249 2665 6214
USER 13563 RECOMMENDED!
11342 13443 6431 4556 12111 2665 12901 14358 6527 9993
USER 13564 RECOMMENDED!
5925 7716 10562 6214 3942 14627 15778 4088 12999 12457
USER 13565 RECOMMENDED!
1941 5273 1551 8326 12186 1259 2152 5925 13178 17584
USER 13566 RECOMMENDED!
8635 13134 16365 15082 13178 17364 3768 241 4607 16791
USER 13567 RECOMMENDED!
13630 15289 16846 14358 17330 7257 10865 13950 5925 13229
USER 13568 RECOMMENDED!
13134 6177 4088 17748 1941 5424 9744 4078 16365 4853
USER 13569 RECOMMENDED!
17356 12901 6589 14078 1614 5273 17584 13914 12186 14229
USER 13570 RECOMMENDED!
6177 3899 12707 8942 14449 6540 14762 9871 3204 16846
USER 13571 RECOMMENDED!
13134 14181 8194 10562 12492 153